In [35]:
# Initialize the random number generator
rng = MersenneTwister(2016);

In [36]:
function pretty_print(A)
    m = size(A,1)
    n = size(A,2)
    for i=1:m
        for j=1:n
            @printf "%12.3e" A[i,j]
        end
        @printf "\n"
    end
end    

pretty_print (generic function with 1 method)

In [37]:
# Size of matrix
n = 3
X = rand(rng, n, n)
Λ = diagm(Float32[i for i=1:n])
A = X * Λ / X

3x3 Array{Float64,2}:
 4.30615  -10.9092   6.57564
 1.44662   -5.89854  4.80889
 1.76282   -9.21215  7.5924 

In [38]:
srand(rng, 16)
Qk = rand(rng, n, n)

for k=1:40
    Qk = A * Qk
    Qk, Rk = qr(Qk)
    if k%10 == 0 || k==1
        println("\nIteration ",k)
        pretty_print(Qk' * A * Qk)
    end
end


Iteration 1
   6.270e+00  -5.469e+00  -1.748e+01
   1.567e-01   1.823e+00  -1.132e+00
   9.756e-01  -1.245e+00  -2.093e+00

Iteration 10
   2.993e+00  -1.242e+00  -1.914e+01
  -5.651e-03   2.007e+00  -2.125e+00
   7.727e-06   2.126e-05   9.999e-01

Iteration 20
   3.000e+00  -1.237e+00  -1.913e+01
  -9.966e-05   2.000e+00  -2.232e+00
   1.317e-10   2.982e-08   1.000e+00

Iteration 30
   3.000e+00  -1.237e+00  -1.913e+01
  -1.729e-06   2.000e+00  -2.234e+00
   2.331e-15   2.927e-11   1.000e+00

Iteration 40
   3.000e+00  -1.237e+00  -1.913e+01
  -2.998e-08   2.000e+00  -2.234e+00
   7.216e-16   2.826e-14   1.000e+00


In [39]:
n = 8; srand(rng, 16)
X = rand(rng, n, n); Λ = diagm(Float32[i for i=1:n]); 
A = X * Λ / X
niter = 40
Qk = rand(rng, n, n); ev = zeros(niter,n)
for k=1:niter
    Qk = A * Qk
    Qk, Rk = qr(Qk)
    ev[k,:] = diag(Qk' * A * Qk)
end

In [40]:
using Plots
plotlyjs()
plot(ev[:,1],lab=string("entry ",1))
for i=2:n-1
     plot!(ev[:,i],lab=string("entry ",i))
end
plot!(ev[:,n],lab=string("entry ",n)) # Small bug: we need to end this cell with plot!

In [41]:
generate_tex_pgfplot = false
if generate_tex_pgfplot
    eval(Expr(:import,:PGFPlots))
    n = 8; srand(rng, 16)
    X = rand(rng, n, n); Λ = diagm(Float32[i for i=1:n]); A = X * Λ / X
    niter = 30
    Qk = rand(rng, n, n); ev = zeros(niter,n)
    for k=1:niter
        Qk = A * Qk
        Qk, Rk = qr(Qk)
        ev[k,:] = diag(Qk' * A * Qk)
    end

    traces = PGFPlots.Plots.Linear[]
    for i=1:n
        traces = [traces; PGFPlots.Plots.Linear(1:niter,ev[:,i],legendentry=string("entry ",i))]
    end
    ax = PGFPlots.Axis(traces, xlabel="Iteration", ylabel="Diagonal entry")
    ax.legendStyle = "{at={(1.05,1.0)},anchor=north west}"
    PGFPlots.save("schur_iteration.tex", ax, include_preamble=false)
end